In [2]:
from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, when, regexp_extract, udf, lit, sum, count, desc, mean, explode, trim, split
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, LongType
import re
import os
import dotenv
dotenv.load_dotenv(".env")

False

In [3]:
spark = SparkSession.builder.appName("IMDB_Movies_Analysis").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/23 15:23:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# dataset link https://www.kaggle.com/datasets/hetbabariya/imdb-movies-data-collection-5000-records?resource=download
# spark is not the best for csv. If csv is really bad, you will have a problem
df = spark.read.csv("IMDB_Movies_Dataset.csv", header=True, inferSchema=True)
df.show()

+---+--------------------+--------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+
|_c0|               Title|Average Rating|            Director|              Writer|Metascore|                Cast|        Release Date|   Country of Origin|           Languages|              Budget|Worldwide Gross|           Runtime|
+---+--------------------+--------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+
|  0|The Shawshank Red...|           9.3|      Frank Darabont|Stephen King, Fra...|     82.0|Tim Robbins, Morg...|October 14, 1994 ...|       United States|             English|$25,000,000 (esti...|    $29,332,133|2 hours 22 minutes|
|  1|Attack on Titan t...|           9.3|    Yûichirô Hayashi|Na

25/01/23 15:24:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Title, Average Rating, Director, Writer, Metascore, Cast, Release Date, Country of Origin, Languages, Budget, Worldwide Gross, Runtime
 Schema: _c0, Title, Average Rating, Director, Writer, Metascore, Cast, Release Date, Country of Origin, Languages, Budget, Worldwide Gross, Runtime
Expected: _c0 but found: 
CSV file: file:///Users/apple/IdeaProjects/Knowledge-sharing/tools/Pandas%20vs%20Polars/IMDB_Movies_Dataset.csv


In [5]:
df = df.drop("_c0")
df.show()

+--------------------+--------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+
|               Title|Average Rating|            Director|              Writer|Metascore|                Cast|        Release Date|   Country of Origin|           Languages|              Budget|Worldwide Gross|           Runtime|
+--------------------+--------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+
|The Shawshank Red...|           9.3|      Frank Darabont|Stephen King, Fra...|     82.0|Tim Robbins, Morg...|October 14, 1994 ...|       United States|             English|$25,000,000 (esti...|    $29,332,133|2 hours 22 minutes|
|Attack on Titan t...|           9.3|    Yûichirô Hayashi|Natsuki Hanae, Yo...| 

In [6]:
df = df.withColumnRenamed("Average Rating", "Rating")
print(df.columns)


['Title', 'Rating', 'Director', 'Writer', 'Metascore', 'Cast', 'Release Date', 'Country of Origin', 'Languages', 'Budget', 'Worldwide Gross', 'Runtime']


In [7]:
df.printSchema()
df.schema

root
 |-- Title: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Director: string (nullable = true)
 |-- Writer: string (nullable = true)
 |-- Metascore: double (nullable = true)
 |-- Cast: string (nullable = true)
 |-- Release Date: string (nullable = true)
 |-- Country of Origin: string (nullable = true)
 |-- Languages: string (nullable = true)
 |-- Budget: string (nullable = true)
 |-- Worldwide Gross: string (nullable = true)
 |-- Runtime: string (nullable = true)


StructType([StructField('Title', StringType(), True), StructField('Rating', DoubleType(), True), StructField('Director', StringType(), True), StructField('Writer', StringType(), True), StructField('Metascore', DoubleType(), True), StructField('Cast', StringType(), True), StructField('Release Date', StringType(), True), StructField('Country of Origin', StringType(), True), StructField('Languages', StringType(), True), StructField('Budget', StringType(), True), StructField('Worldwide Gross', StringType(), True), StructField('Runtime', StringType(), True)])

In [8]:
(df
 .select([(count("*") - count(c)).alias(c) for c in df.columns])
 .show()
 )

+-----+------+--------+------+---------+----+------------+-----------------+---------+------+---------------+-------+
|Title|Rating|Director|Writer|Metascore|Cast|Release Date|Country of Origin|Languages|Budget|Worldwide Gross|Runtime|
+-----+------+--------+------+---------+----+------------+-----------------+---------+------+---------------+-------+
|    0|     0|       0|     1|     1934|   7|           0|                3|       21|  2338|           1094|      0|
+-----+------+--------+------+---------+----+------------+-----------------+---------+------+---------------+-------+


In [9]:
(df
.select([(count("*") - count(c)).alias(c) for c in df.columns])
.explain(extended=True)
)

== Parsed Logical Plan ==
'Project [(count(1) - count('Title)) AS Title#357, (count(1) - count('Rating)) AS Rating#360, (count(1) - count('Director)) AS Director#363, (count(1) - count('Writer)) AS Writer#366, (count(1) - count('Metascore)) AS Metascore#369, (count(1) - count('Cast)) AS Cast#372, (count(1) - count('Release Date)) AS Release Date#375, (count(1) - count('Country of Origin)) AS Country of Origin#378, (count(1) - count('Languages)) AS Languages#381, (count(1) - count('Budget)) AS Budget#384, (count(1) - count('Worldwide Gross)) AS Worldwide Gross#387, (count(1) - count('Runtime)) AS Runtime#390]
+- Project [Title#18, Average Rating#19 AS Rating#183, Director#20, Writer#21, Metascore#22, Cast#23, Release Date#24, Country of Origin#25, Languages#26, Budget#27, Worldwide Gross#28, Runtime#29]
   +- Project [Title#18, Average Rating#19, Director#20, Writer#21, Metascore#22, Cast#23, Release Date#24, Country of Origin#25, Languages#26, Budget#27, Worldwide Gross#28, Runtime#29]

In [10]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
df.describe().toPandas()

25/01/23 15:24:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,summary,Title,Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime
0,count,4989,4989,4989,4988,3055,4982,4989,4986,4968,2651,3895,4989
1,mean,955.6923076923077,7.632892363198688,None,None,74.68445171849427,None,None,None,None,None,None,None
2,stddev,955.931952304084,0.3652432599971371,None,None,12.298910502972529,None,None,None,None,None,None,None
3,min,"""Mujeres al borde de un ataque de """"nervios""""""",7.2,A.L. Vijay,"A.A. Milne, Ernest Shepard, Alex Ross Perry",25.0,"'Freeway' Ricky Ross, Susan Sarandon, David Si...",1928 (Italy),"Afghanistan, Ireland, Japan, Netherlands, Iran","Aboriginal, Japanese, Hokkien","$1,000,000 (estimated)","$1,003",1 hour
4,max,色，戒,9.3,Özer Kiziltan,Ümit Ünal,100.0,"Éric Cantona, Alex Ferguson, Manchester United...","September 9, 2020 (India)","Yugoslavia, United States","Zulu, Sotho, Xhosa, Afrikaans, English","₹991,000 (estimated)","₹750,000,000",9 hours 26 minutes


In [11]:
df.summary().toPandas()

,summary,Title,Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime
0,count,4989,4989,4989,4988,3055,4982,4989,4986,4968,2651,3895,4989
1,mean,955.6923076923077,7.632892363198688,None,None,74.68445171849427,None,None,None,None,None,None,None
2,stddev,955.931952304084,0.3652432599971371,None,None,12.298910502972529,None,None,None,None,None,None,None
3,min,"""Mujeres al borde de un ataque de """"nervios""""""",7.2,A.L. Vijay,"A.A. Milne, Ernest Shepard, Alex Ross Perry",25.0,"'Freeway' Ricky Ross, Susan Sarandon, David Si...",1928 (Italy),"Afghanistan, Ireland, Japan, Netherlands, Iran","Aboriginal, Japanese, Hokkien","$1,000,000 (estimated)","$1,003",1 hour
4,25%,42.0,7.3,None,None,67.0,None,None,None,None,None,None,None
5,50%,303.0,7.6,None,None,76.0,None,None,None,None,None,None,None
6,75%,1917.0,7.8,None,None,84.0,None,None,None,None,None,None,None
7,max,色，戒,9.3,Özer Kiziltan,Ümit Ünal,100.0,"Éric Cantona, Alex Ferguson, Manchester United...","September 9, 2020 (India)","Yugoslavia, United States","Zulu, Sotho, Xhosa, Afrikaans, English","₹991,000 (estimated)","₹750,000,000",9 hours 26 minutes


In [12]:
(df
 .groupBy("Runtime")
 .count()
 .orderBy(desc("count")) 
 .orderBy(col("count").desc()) 
 .show()
 )

+-----------------+-----+
|          Runtime|count|
+-----------------+-----+
|1 hour 50 minutes|  109|
|1 hour 35 minutes|  109|
|1 hour 43 minutes|  100|
|1 hour 58 minutes|   94|
|1 hour 42 minutes|   92|
|1 hour 30 minutes|   92|
|1 hour 40 minutes|   92|
|1 hour 52 minutes|   91|
|          2 hours|   91|
|1 hour 45 minutes|   90|
|1 hour 44 minutes|   90|
|1 hour 36 minutes|   89|
|1 hour 48 minutes|   89|
|1 hour 46 minutes|   87|
|1 hour 53 minutes|   86|
|1 hour 37 minutes|   85|
|1 hour 56 minutes|   84|
|1 hour 41 minutes|   82|
|1 hour 39 minutes|   81|
|1 hour 47 minutes|   80|
+-----------------+-----+


In [13]:
def runtime_to_minutes(string: str) -> int:
    hours = re.search(r"(\d+) hours?", string)
    if hours:
        hours = int(hours.group(1))
    else:
        hours = 0
    minutes = re.search(r"(\d+) minutes", string)
    if minutes:
        minutes = int(minutes.group(1))
    else:
        minutes = 0
    return hours * 60 + minutes


runtime_to_minutes("2 hours 25 minutes")

145

In [14]:
runtime_udf = udf(runtime_to_minutes, IntegerType())

In [15]:
(
    df
    .withColumn("Runtime (minutes)", runtime_udf("Runtime"))
    .withColumn("hours", regexp_extract("Runtime", "(\\d+) hour", 1).cast("int"))
    .withColumn("minutes", regexp_extract("Runtime", "(\\d+) minutes", 1).cast("int"))
    .fillna({"hours": 0, "minutes": 0}) # I would prefer to use coalesce(regexp_extract("Runtime", "(\\d+) minutes", 1).cast("int"), 0)
    .withColumn("Runtime (minutes) v2", col("hours") * 60 + col("minutes"))
    .show()
    
)

+--------------------+------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+-----------------+-----+-------+--------------------+
|               Title|Rating|            Director|              Writer|Metascore|                Cast|        Release Date|   Country of Origin|           Languages|              Budget|Worldwide Gross|           Runtime|Runtime (minutes)|hours|minutes|Runtime (minutes) v2|
+--------------------+------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+-----------------+-----+-------+--------------------+
|The Shawshank Red...|   9.3|      Frank Darabont|Stephen King, Fra...|     82.0|Tim Robbins, Morg...|October 14, 1994 ...|       United States|             English|$25,000,00

In [16]:
def money_to_usd(string: str | None) -> int:
    if not isinstance(string, str) or not "$" in string:
        return None
    budget = int(re.sub(r"[^\d]", "", string))
    return budget
money_to_usd("$6,000,000 (estimated)")

6000000

In [17]:
money_udf = udf(money_to_usd, LongType()) # if use IntegerType it will work incorrect

# You can use withColumns() with dict instead of multiple withColumn
df \
    .withColumn("Budget (usd)", money_udf("Budget")) \
    .withColumn("Worldwide Gross (usd)", money_udf("Worldwide Gross")) \
    .withColumn("Profit (usd)", col("Worldwide Gross (usd)") - col("Budget (usd)")) \
    .show()



+--------------------+------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+------------+---------------------+------------+
|               Title|Rating|            Director|              Writer|Metascore|                Cast|        Release Date|   Country of Origin|           Languages|              Budget|Worldwide Gross|           Runtime|Budget (usd)|Worldwide Gross (usd)|Profit (usd)|
+--------------------+------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+------------+---------------------+------------+
|The Shawshank Red...|   9.3|      Frank Darabont|Stephen King, Fra...|     82.0|Tim Robbins, Morg...|October 14, 1994 ...|       United States|             English|$25,000,000 (esti...|    

In [18]:
money_df = df \
    .withColumn("budget_usd", money_udf("Budget")) \
    .withColumn("ww_gross_usd", money_udf("Worldwide Gross"))

money_df \
    .withColumn("profit", money_df.ww_gross_usd - money_df.budget_usd) \
    .show()
# can be useful for joins

+--------------------+------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+----------+------------+----------+
|               Title|Rating|            Director|              Writer|Metascore|                Cast|        Release Date|   Country of Origin|           Languages|              Budget|Worldwide Gross|           Runtime|budget_usd|ww_gross_usd|    profit|
+--------------------+------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+----------+------------+----------+
|The Shawshank Red...|   9.3|      Frank Darabont|Stephen King, Fra...|     82.0|Tim Robbins, Morg...|October 14, 1994 ...|       United States|             English|$25,000,000 (esti...|    $29,332,133|2 hours 22 minutes|  250000

In [19]:
highest_budget = money_df.orderBy(desc("budget_usd")).limit(10)
highest_budget.show()

+--------------------+------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+----------+------------+
|               Title|Rating|            Director|              Writer|Metascore|                Cast|        Release Date|   Country of Origin|           Languages|              Budget|Worldwide Gross|           Runtime|budget_usd|ww_gross_usd|
+--------------------+------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+----------+------------+
|   Avengers: Endgame|   8.4|Anthony Russo, Jo...|Christopher Marku...|     78.0|Robert Downey Jr....|April 26, 2019 (I...|       United States|English, Japanese...|$356,000,000 (est...| $2,799,439,100|  3 hours 1 minute| 356000000|  2799439100|
|  The Little Me

In [20]:
# want to plot? Use df.toPandas.plot or switch to Zeppeline or Databricks notebooks instead

In [21]:
# df.filter(col("Cast").contains("Johnny Depp")).show()
# df.where(col("Cast").contains("Johnny Depp")).show() # where sounds more logical, no? With filter you don't understand if you are keeping them or filtering out

df.filter(df.Cast.contains("Johnny Depp")).show(20, False)


+------------------------------------------------------+------+------------------------+----------------------------------------------------+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------+--------------------------------------+-----------------------------------------+------------------------+---------------+------------------+
|Title                                                 |Rating|Director                |Writer                                              |Metascore|Cast                                                                                                                                                                                                                              

In [22]:
# groupBy returns not a DataFrame but specific type of DataFrame, so after it you can use only aggregations
money_df.groupBy("Director") \
    .agg(sum("ww_gross_usd").alias("total_gross")) \
    .orderBy(desc("total_gross")) \
    .limit(10) \
    .show()

+--------------------+-----------+
|            Director|total_gross|
+--------------------+-----------+
|       James Cameron| 8705319635|
|    Steven Spielberg| 7988558298|
|Anthony Russo, Jo...| 6721322058|
|       Peter Jackson| 6484962056|
|   Christopher Nolan| 6205459936|
|         David Yates| 5003353447|
|           Jon Watts| 3966372984|
|         Joss Whedon| 2966001713|
|         J.J. Abrams| 2924357232|
|     Robert Zemeckis| 2793892268|
+--------------------+-----------+


In [23]:
final_df = df.withColumn("Year", regexp_extract(col("Release Date"), r"(\d{4})", 1).cast(IntegerType())) \
    .withColumn("Decade", (col("Year") / 10).cast(IntegerType()) * 10) \

final_df.show()

+--------------------+------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+----+------+
|               Title|Rating|            Director|              Writer|Metascore|                Cast|        Release Date|   Country of Origin|           Languages|              Budget|Worldwide Gross|           Runtime|Year|Decade|
+--------------------+------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+----+------+
|The Shawshank Red...|   9.3|      Frank Darabont|Stephen King, Fra...|     82.0|Tim Robbins, Morg...|October 14, 1994 ...|       United States|             English|$25,000,000 (esti...|    $29,332,133|2 hours 22 minutes|1994|  1990|
|Attack on Titan t...|   9.3|    Yûichirô Hayashi|Natsuki Hanae,

In [25]:
final_df.write.mode("overwrite").partitionBy("Decade").parquet("spark_parquet")
# fails if path already exists if you don't specify mode

In [26]:
final_df.createOrReplaceTempView("final_view")
spark.sql("select * from final_view limit 10").show()

+--------------------+------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+----+------+
|               Title|Rating|            Director|              Writer|Metascore|                Cast|        Release Date|   Country of Origin|           Languages|              Budget|Worldwide Gross|           Runtime|Year|Decade|
+--------------------+------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+----+------+
|The Shawshank Red...|   9.3|      Frank Darabont|Stephen King, Fra...|     82.0|Tim Robbins, Morg...|October 14, 1994 ...|       United States|             English|$25,000,000 (esti...|    $29,332,133|2 hours 22 minutes|1994|  1990|
|Attack on Titan t...|   9.3|    Yûichirô Hayashi|Natsuki Hanae,

In [30]:
final_df.write.mode("overwrite").saveAsTable("final_table")
spark.sql("select * from final_table limit 10").show()


+--------------------+------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+----+------+
|               Title|Rating|            Director|              Writer|Metascore|                Cast|        Release Date|   Country of Origin|           Languages|              Budget|Worldwide Gross|           Runtime|Year|Decade|
+--------------------+------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+----+------+
|The Shawshank Red...|   9.3|      Frank Darabont|Stephen King, Fra...|     82.0|Tim Robbins, Morg...|October 14, 1994 ...|       United States|             English|$25,000,000 (esti...|    $29,332,133|2 hours 22 minutes|1994|  1990|
|Attack on Titan t...|   9.3|    Yûichirô Hayashi|Natsuki Hanae,

In [ ]:
# to read from db use jdbc
# but you will need to install JDBC driver for your particular database first


# Window functions

In [58]:

from pyspark.sql import Window

# Task: For each language find the maximum difference in years between two movies. Return names and dates of both movies
window_spec = Window.partitionBy("language").orderBy("date")

# instead of raising an error when date pattern doesn't match just return null
# why it is a separate config but not a function param?
spark.conf.set("spark.sql.legacy.timeParserPolicy", "CORRECTED")

# Go step-by-step

diff_df = (df
           .withColumn("languages_lst", split(df.Languages, ","))
           .withColumn("language", explode("languages_lst"))
           .withColumn("language", trim("language"))
           .filter(col("language").isNotNull())
           .withColumn("date_cleaned", trim(regexp_replace(col("Release Date"), "\\(.+\\)", "")))
           .withColumn("date_v1", to_date(col("date_cleaned"), "MMMM d, yyyy"))
           .withColumn("date_v2", to_date(col("date_cleaned"), "yyyy"))
           .withColumn("date_v3", to_date(col("date_cleaned"), "MMMM yyyy"))
           .withColumn("date", coalesce("date_v1", "date_v2", "date_v3"))
           # .select("Release Date", "date_cleaned", "date_v1", "date_v2", "date_v3", "date")
           # .filter(col("date").isNull())
           .withColumn("previous_movie_date", lag("date", 1).over(window_spec))
           .withColumn("previous_movie_title", lag("Title", 1).over(window_spec))

           # .select("language", "date", "previous_movie_date")
           .filter(col("previous_movie_date").isNotNull())
           .withColumn("previous_movie_diff", datediff(col("date"), col("previous_movie_date")))
           .withColumn("previous_movie_diff_years", round(col("previous_movie_diff") / 365))
           .withColumn("rn", row_number().over(Window.partitionBy("language").orderBy(desc("previous_movie_diff"))))
           .filter(col("rn") == 1)
           .select(col("Title").alias("title"), "date", "previous_movie_title", "previous_movie_date",
                   "previous_movie_diff", "previous_movie_diff_years", "language", "languages")
           )

(
    diff_df
    .orderBy("language")
    .show(500, False)
)



+------------------------------------------------+----------+-------------------------------------------------+-------------------+-------------------+-------------------------+----------------------+----------------------------------------------------------------------------------------------------------+
|title                                           |date      |previous_movie_title                             |previous_movie_date|previous_movie_diff|previous_movie_diff_years|language              |languages                                                                                                 |
+------------------------------------------------+----------+-------------------------------------------------+-------------------+-------------------+-------------------------+----------------------+----------------------------------------------------------------------------------------------------------+
|Rabbit-Proof Fence                              |2002-02-21|Walkabout      

In [56]:
diff_df.filter(diff_df.language.contains("Ukr")).show(500, False)

+---------+----------+-------------------------+-------------------+-------------------+-------------------------+---------+---------------------------+
|title    |date      |previous_movie_title     |previous_movie_date|previous_movie_diff|previous_movie_diff_years|language |languages                  |
+---------+----------+-------------------------+-------------------+-------------------+-------------------------+---------+---------------------------+
|Brother 2|2000-05-11|V Boy Idut Odni 'stariki'|1974-08-12         |9404               |26.0                     |Ukrainian|Russian, English, Ukrainian|
+---------+----------+-------------------------+-------------------+-------------------+-------------------------+---------+---------------------------+


In [57]:
# double check
df.filter(df.Languages.contains("Aboriginal")).show(20, False)

+-------------------------------+------+-------------+----------------------------------------------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------+----------------------------------------+----------------------------------------------------------------+-----------------------+---------------+------------------+
|Title                          |Rating|Director     |Writer                                              |Metascore|Cast                                                                                                                                                                                                                                                                          

True